# 🦌 Make the ELK App Interactive 🕹️

This notebook shows how you can make the ELK App work dynamically with various types of
graphs

In [ ]:
import random
from collections import defaultdict
from pprint import pprint

import importnb
import ipywidgets as W
import networkx as nx
from IPython.display import JSON, display

import ipyelk
from ipyelk import nx as elknx

In [ ]:
with importnb.Notebook():
    from __03_App import a_more_stylish_elk_app_example

In [ ]:
def make_random_forest(number_of_nodes, hierarchy_roots=1, seed=None):
    """ Create a random directed graph that meets NetworkX's forest criteria """
    if seed is not None:
        random.seed(seed)

    if hierarchy_roots < 1:
        return None

    unassigned = set(range(number_of_nodes))
    assigned = set(random.sample(unassigned, hierarchy_roots))
    unassigned -= assigned

    tree_edges = []
    while unassigned:
        node = random.sample(unassigned, 1)[0]
        tree_edges.append((random.sample(assigned, 1)[0], node))
        unassigned -= {node}
        assigned |= {node}

    return nx.DiGraph(tree_edges)

In [ ]:
def find_all(obj, key):
    """ Find all values of a given key in a dictionary"""
    if not isinstance(obj, dict):
        return

    for k, value in obj.items():
        if k == key:
            yield value
        if isinstance(value, dict):
            for result in find_all(value, key):
                yield result
        elif isinstance(value, (tuple, list, set)):
            for item in value:
                for result in find_all(item, key):
                    yield result

In [ ]:
def an_elk_in_a_random_forest():
    app, box, xelk, elk = a_more_stylish_elk_app_example()
    box.layout.flex = "1"
    box.layout.height = "100%"
    out = W.Output()

    with out:

        @W.interact
        def make_graph(
            number_of_nodes=(5, 20),
            percent_of_edges=(1, 99),
            hierarchy_roots=(0, 5),
            seed=(0, 1024),
            debug=False,
            fit=False,
            padding=(0, 100),
            port_scale=(1, 20),
            text_scale=(1, 20),
            label_offset=(1, 10),
        ):
            hierarchy = make_random_forest(
                number_of_nodes=number_of_nodes,
                hierarchy_roots=hierarchy_roots,
                seed=seed,
            )

            number_of_edges = max(1, int(number_of_nodes * 0.01 * percent_of_edges))

            graph = nx.generators.random_graphs.barabasi_albert_graph(
                n=number_of_nodes,
                m=number_of_edges,
                seed=seed,
            )

            for edge in sorted(graph.edges):
                graph.edges[edge]["sourcePort"] = edge
                graph.edges[edge]["targetPort"] = edge

            with app.transformer.hold_trait_notifications():
                app.transformer.port_scale = port_scale
                app.transformer.label_offset = label_offset
                app.transformer.text_scale = text_scale
                app.transformer.source = (graph, hierarchy)

            if fit:
                app.diagram.fit("root", padding=padding)

            if debug:
                counter = defaultdict(list)
                [counter[src].append(tgt) for src, tgt in graph.edges]
                pprint(dict(counter))
                pprint(sum(list(find_all(elk_app.transformer.value, "edges")), []))

            out.clear_output()
            return JSON(elk_app.diagram.value)

    out.clear_output()

    return (
        app,
        W.HBox(
            [W.VBox([make_graph.widget]), box], layout=dict(height="100%", flex="1")
        ),
        xelk,
        elk,
    )

In [ ]:
if __name__ == "__main__":
    display(an_elk_in_a_random_forest()[1])

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).